In [17]:
!pip install langchain chromadb openai tiktoken pypdf langchain_openai langchain-community

In [18]:
!pip install langchain_groq

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document


In [20]:
doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [21]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [22]:
vector_store = Chroma(
    embedding_function=HuggingFaceEmbeddings(),
    persist_directory='my_chroma_db',
    collection_name='cricket'
)

/tmp/ipython-input-382736866.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_function=HuggingFaceEmbeddings(),


In [23]:
#add documents
vector_store.add_documents(docs)

['a7119efc-86e9-41a6-aa1a-b95decb10888',
 'adf78cd6-12d6-4cf0-917d-8644067743b5',
 '6e52d846-9836-47c7-90f9-f410994a9b70',
 '3901cdb1-c9f7-4664-bb90-96725149cb21',
 '0d91aad9-e748-410b-810e-3e1bb9574088']

In [24]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['a7119efc-86e9-41a6-aa1a-b95decb10888',
  'adf78cd6-12d6-4cf0-917d-8644067743b5',
  '6e52d846-9836-47c7-90f9-f410994a9b70',
  '3901cdb1-c9f7-4664-bb90-96725149cb21',
  '0d91aad9-e748-410b-810e-3e1bb9574088'],
 'embeddings': array([[-0.03964761, -0.00801649, -0.01412725, ...,  0.06814166,
         -0.00116097, -0.02425426],
        [-0.01992596,  0.00161874, -0.00959343, ...,  0.02433112,
         -0.00341396, -0.01722028],
        [-0.0425618 ,  0.01060331, -0.01305767, ...,  0.04738369,
         -0.01748697,  0.00204662],
        [-0.02734396, -0.03300869,  0.01565378, ...,  0.01462703,
          0.01177981,  0.00556419],
        [ 0.02301688, -0.02011727, -0.00123582, ...,  0.02533475,
          0.00280527, -0.0512028 ]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [26]:
#search documents
vector_store.similarity_search(
    query='who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
 Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.')]

In [29]:
#search with similarity score
vector_store.similarity_search_with_score(
    query='who is the most successfull captian?',
    k=2
)

[(Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'),
  1.1610867977142334),
 (Document(metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.1843235492706299)]

In [30]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.8879573345184326),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  2.0017194747924805)]

In [31]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='a7119efc-86e9-41a6-aa1a-b95decb10888', document=updated_doc1)


In [32]:
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['a7119efc-86e9-41a6-aa1a-b95decb10888',
  'adf78cd6-12d6-4cf0-917d-8644067743b5',
  '6e52d846-9836-47c7-90f9-f410994a9b70',
  '3901cdb1-c9f7-4664-bb90-96725149cb21',
  '0d91aad9-e748-410b-810e-3e1bb9574088'],
 'embeddings': array([[-0.04108338, -0.02528017, -0.0214507 , ...,  0.02231626,
         -0.02287274, -0.00913005],
        [-0.01992596,  0.00161874, -0.00959343, ...,  0.02433112,
         -0.00341396, -0.01722028],
        [-0.0425618 ,  0.01060331, -0.01305767, ...,  0.04738369,
         -0.01748697,  0.00204662],
        [-0.02734396, -0.03300869,  0.01565378, ...,  0.01462703,
          0.01177981,  0.00556419],
        [ 0.02301688, -0.02011727, -0.00123582, ...,  0.02533475,
          0.00280527, -0.0512028 ]]),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a sin

In [33]:
#delete document
vector_store.delete(
    ids=['a7119efc-86e9-41a6-aa1a-b95decb10888']
)

In [34]:
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['adf78cd6-12d6-4cf0-917d-8644067743b5',
  '6e52d846-9836-47c7-90f9-f410994a9b70',
  '3901cdb1-c9f7-4664-bb90-96725149cb21',
  '0d91aad9-e748-410b-810e-3e1bb9574088'],
 'embeddings': array([[-0.01992596,  0.00161874, -0.00959343, ...,  0.02433112,
         -0.00341396, -0.01722028],
        [-0.0425618 ,  0.01060331, -0.01305767, ...,  0.04738369,
         -0.01748697,  0.00204662],
        [-0.02734396, -0.03300869,  0.01565378, ...,  0.01462703,
          0.01177981,  0.00556419],
        [ 0.02301688, -0.02011727, -0.00123582, ...,  0.02533475,
          0.00280527, -0.0512028 ]]),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah is considered one 